# Overview
This notebook will show a basic example of how to use the `Sectionizer` package from medspaCy.

## Prerequisites
This notebook will also use some examples from the master medSpaCy package [medspacy](https://github.com/medspacy/medspacy), which you can download as:

`pip install medspacy`

## Example text
We'll process this document below:

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../..")

In [2]:
with open("../discharge_summary.txt") as f:
    text = f.read()

In [3]:
print(text[:500])

Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]

Date of Birth:  [**2498-8-19**]             Sex:   F

Service: SURGERY

Allergies:
Hydrochlorothiazide

Attending:[**First Name3 (LF) 1893**]
Chief Complaint:
Abdominal pain

Major Surgical or Invasive Procedure:
PICC line [**6-25**]
ERCP w/ sphincterotomy [**5-31**]


History of Present Illness:
74y female with type 2 dm and a recent stroke affecting her
speech, who presents with 2 days of abdominal pain. Imaging sh


# Getting started
The `Sectionizer` component is used in the same way as any other spaCy component. We'll start by loading a spaCy model, adding a few rules for extracting entities, creating a `Sectionizer` object, and then adding it to our pipeline.

In [7]:
import medspacy

In [14]:
nlp = medspacy.load()

In [15]:
nlp.pipe_names

['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']

In [16]:
sectionizer = nlp.add_pipe("medspacy_sectionizer")

In [17]:
nlp.pipe_names

['medspacy_pyrush',
 'medspacy_target_matcher',
 'medspacy_context',
 'medspacy_sectionizer']

In [21]:
# Add some target rules for extracting entities
from medspacy.target_matcher import TargetRule

rules = [
    TargetRule("stroke", "CONDITION"),
    TargetRule("alcohol", "CONDITION"),
    TargetRule("drug use", "CONDITION"),
    TargetRule("Miconazole Nitrate", "MEDICATION"),
    TargetRule("Type 2 DM", "CONDITION")
    
]

In [22]:
nlp.get_pipe("medspacy_target_matcher").add(rules)

## Processing text
As an example, we'll process this text from MIMIC-II:

In [23]:
doc = nlp(text)

/Users/alecchapman/Code/medspacy/medspacy/notebooks/section_detection/../../medspacy/common/medspacy_matcher.py:80: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc)
/Users/alecchapman/Code/medspacy/medspacy/notebooks/section_detection/../../medspacy/common/medspacy_matcher.py:80: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc)


Just like that, we've processed our doc with medSpaCy! Let's first take a look at what entities we've extracted, as well as the section headers. To do this, we'll use a visualizer in the medspaCy package. It is an extension of displaCy that supports medspaCy components.. This function will highlight all of the **entities** found in `doc.ents` which were extracted by our model's `ner` component, as well as the section headers extracted by `sectionizer`:

In [24]:
from medspacy.visualization import visualize_ent

In [25]:
visualize_ent(doc)

The section titles are highlighted in gray with **<<>>** symbols around the normalized section title. As you can see, there are sometimes overlap between the targets and section headers, which causes duplicate text to be displayed.

# Extracted Section Information
Let's now see what information was extracted by `sectionizer`. When `sectionizer` process a `doc`, it adds a number of custom attributes at the following levels:
- `Doc`: The entire document
- `Token`: A single token

In spaCy, custom attributes are saved under the `var._` attribute. 

## Doc
Let's first look at all of the `section_categories` which were found in our text. `category` represents the normalized name specified in your rules.

In [26]:
doc._.section_categories

[None,
 'other',
 'allergies',
 'chief_complaint',
 'history_of_present_illness',
 'past_medical_history',
 'social_history',
 'family_history',
 'hospital_course',
 'medications',
 'observation_and_plan',
 'patient_instructions',
 'signature']

Next, let's find the spans of the text which were recognized as `section_titles`. `title` represents the title or header component of a section. This is the part that is matched using the rules provided to the `Sectionizer` component.

In [27]:
doc._.section_titles

[,
 Service:,
 Allergies:,
 Chief Complaint:,
 History of Present Illness:,
 Past Medical History:,
 Social History:,
 Family History:,
 Brief Hospital Course:,
 Discharge Medications:,
 Discharge Diagnosis:,
 Discharge Instructions:,
 Signed electronically by:]

Now, let's look at the `section_bodies`. `body` represents the text in between each `title` that is assigned to the category of the previous matched `title`. Note that the `title` and the `body` do not overlap.

In [28]:
doc._.section_bodies

[Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]
 
 Date of Birth:  [**2498-8-19**]             Sex:   F
 ,
 SURGERY
 ,
 
 Hydrochlorothiazide
 
 Attending:[**First Name3 (LF) 1893**],
 
 Abdominal pain
 
 Major Surgical or Invasive Procedure:
 PICC line [**6-25**]
 ERCP w/ sphincterotomy [**5-31**]
 
 ,
 
 74y female with type 2 dm and a recent stroke affecting her
 speech, who presents with 2 days of abdominal pain. Imaging shows no evidence of metastasis. She is not receiving any chemo.
 ,
 
 1. Colon cancer dx'd in [**2554**], tx'd with hemicolectomy, XRT,
 chemo. Last colonoscopy showed: Last CEA was in the 8 range
 (down from 9)
 2. Type II Diabetes Mellitus
 3. Hypertension
 ,
 
 Married, former tobacco use. No alcohol or drug use.
 ,
 
 Mother with stroke at age 82. no early deaths.
 2 daughters- healthy
 
 ,
 
 Ms. [**Known patient lastname 2004**] was admitted on [**2573-5-30**]. Ultrasound at the time of
 admission demonstrated pancreatic duct 

Finally, the entire section is available through `section_span`. The `span` represents the combined `title` and `body`.

In [29]:
print(doc._.section_spans)

[Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]

Date of Birth:  [**2498-8-19**]             Sex:   F

, Service: SURGERY

, Allergies:
Hydrochlorothiazide

Attending:[**First Name3 (LF) 1893**]
, Chief Complaint:
Abdominal pain

Major Surgical or Invasive Procedure:
PICC line [**6-25**]
ERCP w/ sphincterotomy [**5-31**]


, History of Present Illness:
74y female with type 2 dm and a recent stroke affecting her
speech, who presents with 2 days of abdominal pain. Imaging shows no evidence of metastasis. She is not receiving any chemo.

, Past Medical History:
1. Colon cancer dx'd in [**2554**], tx'd with hemicolectomy, XRT,
chemo. Last colonoscopy showed: Last CEA was in the 8 range
(down from 9)
2. Type II Diabetes Mellitus
3. Hypertension

, Social History:
Married, former tobacco use. No alcohol or drug use.

, Family History:
Mother with stroke at age 82. no early deaths.
2 daughters- healthy


, Brief Hospital Course:
Ms. [**Known patient lastname 20

## Section Objects

The sections are stored as a custom `Section` obect that can also be accessed through the `doc`.

In [30]:
sections = doc._.sections
sections[1]

Section(category=other, title=Service:, body=SURGERY

, parent=None, rule=SectionRule(literal="Service:", category="other", pattern=None, on_match=None, parents=[], parent_required=False))

`Section` objects have a variety of properties:
* `doc`: the doc associated with the section
* `category`: returns the section category
* `title_start`: the index of the first token of the `title`
* `title_end`: the index of the last token of the `title`
* `title_span`: the span of the `title`
* `body_start`: the index of the first token of the `body`
* `body_end`: the index of the last token of the `body`
* `body_span`: the span of the `body`
* `section_span`: the span of the `title`+`body`
* `parent`: the `Section` that was matched as the parent of the current section (the parent-child pairings are covered in a future notebook)
* `rule`: the rule that matched the `title`

## Span
Now, for each entity extracted from the text, you can extract the associated `Section` object. In this example, lets look at the `category` for the `Section` associated with each entity.

In [31]:
for ent in doc.ents[:10]:
    print(ent, ent._.section.category, sep=", ")

type 2 dm, history_of_present_illness
stroke, history_of_present_illness
alcohol, social_history
drug use, social_history
stroke, family_history
Miconazole Nitrate, medications
Type 2 DM, observation_and_plan


Using the `Section`, you can also access the entire section span which contained the ent:

In [32]:
ent = doc.ents[0]
ent._.section.section_span

History of Present Illness:
74y female with type 2 dm and a recent stroke affecting her
speech, who presents with 2 days of abdominal pain. Imaging shows no evidence of metastasis. She is not receiving any chemo.


# Assertion attributes
In clinical NLP, it's important to account for certain attributes about extracted entities, such as whether a concept is **negated** or **historical**. This is handled at a sentence level by [cycontext](https://github.com/medspacy/cycontext), which looks for linguistic modifiers within the same sentence as an entity. However, the section in which a concept occurs can also inform these attributes.

For example, in the example below, we know that:
- **"Pneumonia"** is not current because it occurs in the **Past Medical History**
- **"Penicillin"** and **"Allergies"** are not actually experienced, they're just listed in the allergies section. We call this **hypothetical**
- **"Diabetes** is experienced by someone in the patient's family because it occurs in the **Family History**
- **"Chest pain"** is hypothetical because it occurs in the **Patient Education** section as a hypothetical event



This functionality can be set on with the `add_attrs` argument in the constructor, which by default is `False`. In spaCy 3, this is done by passing a dictionary to the `config` argument:

In [42]:
nlp = medspacy.load()

In [43]:
sectionizer = nlp.add_pipe("medspacy_sectionizer", config={"add_attrs": True})

In [44]:
# Add some target rules for extracting entities
from medspacy.target_matcher import TargetRule

rules = [
    TargetRule("pneumonia", "CONDITION"),
    TargetRule("diabetes", "CONDITION"),
    TargetRule("PE", "CONDITION"),
    TargetRule("chest pain", "CONDITION"),
    TargetRule("Penicillin", "MEDICATION"),
    TargetRule("warfarin", "MEDICATION"),
    
]

nlp.get_pipe("medspacy_target_matcher").add(rules)

In [45]:
text = """
Past Medical History:
pneumonia

Allergies: 
Penicillin

Family History:
Diabetes

Assessment and Plan:
Warfarin for PE

Patient Education:
You have been prescribed with a medication which is known to cause chest pain.
"""

In [46]:
doc = nlp(text)

/Users/alecchapman/Code/medspacy/medspacy/notebooks/section_detection/../../medspacy/common/medspacy_matcher.py:80: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc)
/Users/alecchapman/Code/medspacy/medspacy/notebooks/section_detection/../../medspacy/common/medspacy_matcher.py:80: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc)


In [47]:
visualize_ent(doc)

Each entity will have the following attributes defined under the `ent._` attribute. Each has a default value of `False` which could be set to True by `sectionizer`:

- `is_negated`
- `is_uncertain`
- `is_historical`
- `is_family`
- `is_hypothetical`

Let's iterate through these entities and see which these attributes in these ents.

In [48]:
for ent in doc.ents:
    print(ent, ent._.section.category)
    print("Historical:", ent._.is_historical, "\tFamily:", ent._.is_family, "\tHypothetical:", ent._.is_hypothetical,)
    print()

pneumonia past_medical_history
Historical: True 	Family: False 	Hypothetical: False

Penicillin allergies
Historical: False 	Family: False 	Hypothetical: False

Diabetes family_history
Historical: False 	Family: True 	Hypothetical: False

Warfarin family_history
Historical: False 	Family: True 	Hypothetical: False

PE family_history
Historical: False 	Family: True 	Hypothetical: False

chest pain patient_education
Historical: False 	Family: False 	Hypothetical: False



The attributes and sections are defined in a dictionary mapping the **section titles** to the attribute name/value pairs. You can find this in the `assertion_attributes_mapping` attribute:

In [49]:
sectionizer.assertion_attributes_mapping

{'past_medical_history': {'is_historical': True},
 'sexual_and_social_history': {'is_historical': True},
 'family_history': {'is_family': True},
 'patient_instructions': {'is_hypothetical': True},
 'education': {'is_hypothetical': True},
 'allergy': {'is_hypothetical': True}}

Additionally, you could define your own logic by constructing a dictionary like the one above, registering the `Span` extensions, passing the dictionary in to the `add_attrs` argument:
```python
sectionizer = Sectionizer(nlp, add_attr={...})
```